In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ✅ นำเข้าไลบรารี
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# ✅ กำหนด Path ของรูปภาพ
train_dir = "/content/drive/MyDrive/fruits_dataset/train"
valid_dir = "/content/drive/MyDrive/fruits_dataset/valid"
test_dir = "/content/drive/MyDrive/fruits_dataset/test"

# ✅ โหลด CSV Meta Data
train_df = pd.read_csv("/content/drive/MyDrive/fruits_dataset/train_meta_data.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/fruits_dataset/valid_meta_data.csv")
test_df = pd.read_csv("/content/drive/MyDrive/fruits_dataset/test_meta_data.csv")

# ✅ ลบคอลัมน์ filename ออก (ใช้เฉพาะ Label)
columns = train_df.columns[1:]
train_labels = train_df[columns]
valid_labels = valid_df[columns]
test_labels = test_df[columns]

# ✅ ใช้ ImageDataGenerator เพื่อ Normalize ข้อมูล
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col="filename",
    y_col=columns,
    target_size=(128, 128),
    batch_size=32,
    class_mode="raw"
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory=valid_dir,
    x_col="filename",
    y_col=columns,
    target_size=(128, 128),
    batch_size=32,
    class_mode="raw"
)


Mounted at /content/drive
Found 2697 validated image filenames.
Found 187 validated image filenames.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# ✅ สร้างโมเดล CNN สำหรับ Multi-Label Classification
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),

    Dense(len(columns), activation="sigmoid")  # Multi-Label ใช้ Sigmoid
])

# ✅ Compile โมเดล
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",  # Multi-Label ใช้ Binary Crossentropy
    metrics=["accuracy"]
)

# ✅ แสดงโครงสร้างโมเดล
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 9)                   │           2,313 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,518,345 (24.87 MB)

 Trainable params: 6,518,345 (24.87 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ✅ ใช้ EarlyStopping เพื่อลด Overfitting
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# ✅ Train โมเดล
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    callbacks=[early_stopping]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 805s 9s/step - accuracy: 0.2443 - loss: 0.3540 - val_accuracy: 0.5508 - val_loss: 0.2171
Epoch 2/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.5571 - loss: 0.2303 - val_accuracy: 0.6257 - val_loss: 0.1848
Epoch 3/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.6424 - loss: 0.1932 - val_accuracy: 0.7433 - val_loss: 0.1427
Epoch 4/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.7228 - loss: 0.1584 - val_accuracy: 0.7273 - val_loss: 0.1499
Epoch 5/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.7515 - loss: 0.1479 - val_accuracy: 0.7594 - val_loss: 0.1277
Epoch 6/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.8198 - loss: 0.1123 - val_accuracy: 0.7326 - val_loss: 0.1413
Epoch 7/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.8430 - loss: 0.0958 - val_accuracy: 0.7914 - val_loss: 0.1084
Epoch 8/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.8604 - loss: 0.0871 - val_accuracy: 0.7701 - v

In [ ]:
# ✅ ทดสอบโมเดลกับชุด Test Data
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col="filename",
    y_col=columns,
    target_size=(128, 128),
    batch_size=32,
    class_mode="raw",
    shuffle=False
)

# ✅ ประเมินผลโมเดล
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n🏆 Test Accuracy: {test_acc:.4f}")

# ✅ ทำนายผล
y_pred = model.predict(test_generator)
y_pred_labels = (y_pred > 0.5).astype(int)  # เปลี่ยนค่า Sigmoid เป็น 0 หรือ 1

# ✅ แสดงผลการทดสอบ
from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_labels, target_names=columns))


Found 90 validated image filenames.
